In [1]:
import numba as nb
import numpy as np
from index import*
from env import*

# list_player = [player_random]*8 + [player_input]
list_player = [player_random]*NUMBER_PLAYER
file_temp = [[0]]*NUMBER_PLAYER
file_per = [0]

env_state = initEnv()
env_state = reset_round(env_state)
while env_state[ENV_STATUS_GAME] != 6:
    action, file_temp, file_per = action_player(env_state,list_player,file_temp,file_per)
    env_state = stepEnv(env_state, action)

for id in range(NUMBER_PLAYER):
    id_player = int(id + env_state[ENV_ID_ACTION]) % NUMBER_PLAYER
    if env_state[ENV_ALL_PLAYER_STATUS + id_player] == 0:
        continue
    else:
        action, file_temp, file_per = action_player(env_state,list_player,file_temp,file_per)
        env_state[ENV_ID_ACTION] = (id_player + 1) % NUMBER_PLAYER


In [2]:
count_all, file_per = normal_main(list_player, 1, [0])
count_all

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [4]:
count_all, file_per = normal_main(list_player, 1000000, [0])
count_all

KeyboardInterrupt: 

In [6]:
env_state[ENV_ALL_PLAYER_STATUS : ENV_ALL_FIRST_CARD]

array([1., 1., 1., 1., 0., 1., 1., 0., 0.])

In [7]:
env_state = reset_round(env_state)
while env_state[ENV_STATUS_GAME] != 6:
    action, file_temp, file_per = action_player(env_state,list_player,file_temp,file_per)
    env_state = stepEnv(env_state, action)

for id in range(NUMBER_PLAYER):
    id_player = int(id + env_state[ENV_ID_ACTION]) % NUMBER_PLAYER
    if env_state[ENV_ALL_PLAYER_STATUS + id_player] == 0:
        continue
    else:
        action, file_temp, file_per = action_player(env_state,list_player,file_temp,file_per)
        env_state[ENV_ID_ACTION] = (id_player + 1) % NUMBER_PLAYER

update chip give
200.0 2.0 2.0
[1. 0. 1. 0. 0. 0.]
4 phase 0 fold button 4.0 1.0 status 0 [210. 198. 197. 196. 200. 198. 198. 200. 200.] [0. 0. 1. 2. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1.]
check 5 0.0 4.0
198.0 2.0 2.0
[1. 0. 1. 0. 0. 0.]
5 phase 0 fold button 4.0 1.0 status 0 [210. 198. 197. 196. 200. 198. 198. 200. 200.] [0. 0. 1. 2. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 0. 1. 1. 1. 1.]
check 6 0.0 4.0
198.0 2.0 2.0
[1. 0. 1. 0. 0. 0.]
6 phase 0 fold button 4.0 1.0 status 0 [210. 198. 197. 196. 200. 198. 198. 200. 200.] [0. 0. 1. 2. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 0. 0. 1. 1. 1.]
check 7 0.0 4.0
200.0 2.0 2.0
[1. 0. 1. 0. 0. 0.]
7 phase 0 call button 4.0 1.0 status 0 [210. 198. 197. 196. 200. 198. 198. 200. 200.] [0. 0. 1. 2. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 0. 0. 0. 1. 1.]
check 8 0.0 4.0
200.0 2.0 2.0
[1. 0. 1. 0. 0. 0.]
8 phase 0 fold button 4.0 1.0 status 0 [210. 198. 197. 196. 200. 198. 198. 198. 200.] [0. 0. 1. 2. 0. 0. 0. 2. 0.] [1. 1. 1. 1. 0. 0. 0. 1. 1.]
check 0 0.0 4.0
210.0 2.0 2.0

In [8]:
env_state.astype(np.int64)

array([ 44,   4,  16,   7,  29,  27,  11,  36,   2,  10,  15,   0,  34,
        32,   3,  45,  37,  46,  47,  20,  22,  30,  43,   5,  48,  14,
         1,  21,  41,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
        -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
        25,  12,  38,  31,  19, 208, 196, 197, 196, 200, 198, 198, 207,
       200,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   2,   1,
         2,   0,   0,   0,   2,   0,   1,   1,   0,   1,   0,   0,   0,
         1,   0,  13,  35,  17,  26,  51,  23,   6,   9,  50,  33,   8,
        18,  49,  40,  42,  24,  39,  28,  -1,  -1,  -1,  -1,  -1,  -1,
        -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  39,  -1,   1,
         3,   6,   0,   8,   0,   0,   9,   2], dtype=int64)

In [2]:
@nb.njit()
def step(env_state, action):
    phase_env = env_state[ENV_PHASE]
    id_action = int(env_state[ENV_ID_ACTION])
    if phase_env == 0:
        if action == 0:     #người chơi call
            #trừ chip người chơi và tăng giá trị pot, bổ sung giá trị số tiền đã bỏ ra
            chip_to_call = env_state[ENV_CASH_TO_CALL_OLD] - env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action]
            env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action] += chip_to_call
            env_state[ENV_ALL_PLAYER_CHIP_IN_POT+id_action] += chip_to_call
            env_state[ENV_ALL_PLAYER_CHIP + id_action] -= chip_to_call
            env_state[ENV_POT_VALUE] += chip_to_call
        elif action == 1:   #người chơi check
            pass
        elif action == 2:   #người chơi fold
            #nếu người chơi fold, chuyển trạng thái của người đó.
            env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action] = 0
            env_state[ENV_ALL_PLAYER_STATUS + id_action] = 0
        elif action == 3:   #người chơi bet/raise
            #nếu người chơi bet, trừ tiền người chơi, tăng tiền pot, tăng cash to call_new
            #bổ sung giá trị số tiền đã bỏ ra
            chip_to_bet_raise = 0
            if env_state[ENV_CASH_TO_CALL_OLD] == 0:    #nếu chưa ai bet thì mình bet 
                chip_to_bet_raise = SMALL_CHIP
            else:           #nếu đã có người bet thì mình raise
                chip_to_bet_raise = 2*env_state[ENV_CASH_TO_CALL_OLD] - env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action]

            env_state[ENV_ALL_PLAYER_CHIP + id_action] -= chip_to_bet_raise
            env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action] += chip_to_bet_raise
            env_state[ENV_ALL_PLAYER_CHIP_IN_POT+id_action] += chip_to_bet_raise
            env_state[ENV_POT_VALUE] += chip_to_bet_raise
            env_state[ENV_CASH_TO_CALL_NEW] += env_state[ENV_CASH_TO_CALL_OLD]
            #nếu bet thì sang phase xem có bet tiếp k
            env_state[ENV_TEMP_BUTTON] = id_action
            env_state[ENV_PHASE] = 1
        elif action == 4:
            #nếu người chơi all_in, bổ sung giá trị chip người chơi bỏ ra và tăng pot
            chip_to_allin = env_state[ENV_ALL_PLAYER_CHIP + id_action]
            env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action] += chip_to_allin
            env_state[ENV_ALL_PLAYER_CHIP_IN_POT+id_action] += chip_to_allin
            env_state[ENV_POT_VALUE] += chip_to_allin
            #điều chỉnh cash to call nếu cần, điều chỉnh button nếu người chơi bet nhiều chip nhất
            if env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action] > np.max(env_state[ENV_ALL_PLAYER_CHIP_GIVE : ENV_ALL_PLAYER_CHIP_IN_POT]):
                env_state[ENV_CASH_TO_CALL_OLD] = chip_to_allin
                env_state[ENV_CASH_TO_CALL_NEW] = env_state[ENV_CASH_TO_CALL_OLD]
                env_state[ENV_TEMP_BUTTON] = id_action
            #trừ hết tiền người chơi all in
            env_state[ENV_ALL_PLAYER_CHIP + id_action] = 0

        if action != 3:
            id_action_next = (id_action + 1) % NUMBER_PLAYER
            '''
            hết lượt của người chơi hiện tại, chuyển người chơi, loop đến khi tìm đc người kế tiếp có thể action loop trở về đến
            vị trí hiện tại thì cũng dừng
            '''
            while env_state[ENV_ALL_PLAYER_STATUS + id_action_next] == 0 or env_state[ENV_ALL_PLAYER_CHIP + id_action_next] == 0:
                id_action_next = (id_action_next + 1) % NUMBER_PLAYER
                if id_action_next == env_state[ENV_TEMP_BUTTON]:
                    break
            if id_action_next == env_state[ENV_TEMP_BUTTON]:
                #kết thúc vòng chơi, chuyển status game, reset give chip, xác định id_action mới là người còn chơi và còn chip. Nếu hết 1 vòng => các người chơi đã allin hết, đi thẳng đến showdown
                env_state[ENV_ALL_PLAYER_CHIP_GIVE : ENV_ALL_PLAYER_CHIP_IN_POT] = 0
                env_state[ENV_ID_ACTION] = (env_state[ENV_BUTTON_PLAYER] + 1) % NUMBER_PLAYER
                #loop cho đến khi gặp người vừa còn chơi và vừa còn chip
                while env_state[int(ENV_ALL_PLAYER_STATUS + env_state[ENV_ID_ACTION])] == 0 or env_state[int(ENV_ALL_PLAYER_CHIP + env_state[ENV_ID_ACTION])] == 0:
                    env_state[ENV_ID_ACTION] = (env_state[ENV_ID_ACTION] + 1) % NUMBER_PLAYER
                    #nếu mọi người hết khả năng action tiếp thì đi thẳng đến showdown
                    if env_state[ENV_ID_ACTION] == env_state[ENV_BUTTON_PLAYER]:
                        env_state[ENV_STATUS_GAME] == 6
                        env_state[ENV_ID_ACTION] = env_state[ENV_TEMP_BUTTON]

                if env_state[ENV_STATUS_GAME] == 0:
                    env_state[ENV_STATUS_GAME] = 3
                elif env_state[ENV_STATUS_GAME] == 3:
                    env_state[ENV_STATUS_GAME] = 4
                elif env_state[ENV_STATUS_GAME] == 4:
                    env_state[ENV_STATUS_GAME] = 5
                elif env_state[ENV_STATUS_GAME] == 5:
                    env_state[ENV_STATUS_GAME] == 6


                if env_state[ENV_STATUS_GAME] == 6:
                    env_state = showdown(env_state)
                    env_state[ENV_ID_ACTION] = env_state[ENV_TEMP_BUTTON]
                    
            else:
                #cập nhật người chơi
                env_state[ENV_ID_ACTION] = id_action_next

    elif phase_env == 1:
        if action == 3:
            #nếu người chơi bet tiếp, trừ tiền người chơi, tăng tiền pot, tăng cash to call_new
            #bổ sung giá trị số tiền đã bỏ ra
            chip_to_bet_raise = env_state[ENV_CASH_TO_CALL_OLD]
            env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action] += chip_to_bet_raise
            env_state[ENV_POT_VALUE] += chip_to_bet_raise
            env_state[ENV_CASH_TO_CALL_NEW] += chip_to_bet_raise
            env_state[ENV_ALL_PLAYER_CHIP + id_action] -= chip_to_bet_raise
        elif action == 4:
            #nếu all in khi bet tiếp, update chip give, pot value, cash_to_call_old and cash_to_call_new
            chip_to_allin = env_state[ENV_ALL_PLAYER_CHIP + id_action]
            env_state[ENV_ALL_PLAYER_CHIP_GIVE+id_action] += chip_to_allin
            env_state[ENV_POT_VALUE] += chip_to_allin
            env_state[ENV_CASH_TO_CALL_OLD] = env_state[ENV_CASH_TO_CALL_NEW] + chip_to_allin
            env_state[ENV_CASH_TO_CALL_NEW] = env_state[ENV_CASH_TO_CALL_OLD]
            env_state[ENV_TEMP_BUTTON] = id_action
        elif action == 5:
            env_state[ENV_TEMP_BUTTON] = id_action

        if action != 3:
            '''
            hết lượt của người chơi hiện tại, chuyển người chơi, loop đến khi tìm đc người kế tiếp có thể action loop trở về đến
            vị trí hiện tại thì cũng dừng
            '''
            id_action_next = (id_action + 1) % NUMBER_PLAYER
            while env_state[ENV_ALL_PLAYER_STATUS + id_action_next] == 0 or env_state[ENV_ALL_PLAYER_CHIP + id_action_next] == 0:
                id_action_next = (id_action_next + 1) % NUMBER_PLAYER
                if id_action_next == env_state[ENV_TEMP_BUTTON]:
                    break
            if id_action_next == env_state[ENV_TEMP_BUTTON]:
                #kết thúc vòng chơi, chuyển status game, reset give chip, xác định id_action mới là người còn chơi và còn chip. Nếu hết 1 vòng => các người chơi đã allin hết, đi thẳng đến showdown
                env_state[ENV_ALL_PLAYER_CHIP_GIVE : ENV_ALL_PLAYER_CHIP_IN_POT] = 0
                env_state[ENV_ID_ACTION] = (env_state[ENV_BUTTON_PLAYER] + 1) % NUMBER_PLAYER
                #loop cho đến khi gặp người vừa còn chơi và vừa còn chip
                while env_state[int(ENV_ALL_PLAYER_STATUS + env_state[ENV_ID_ACTION])] == 0 or env_state[int(ENV_ALL_PLAYER_CHIP + env_state[ENV_ID_ACTION])] == 0:
                    env_state[ENV_ID_ACTION] = (env_state[ENV_ID_ACTION] + 1) % NUMBER_PLAYER
                    #nếu mọi người hết khả năng action tiếp thì đi thẳng đến showdown
                    if env_state[ENV_ID_ACTION] == env_state[ENV_BUTTON_PLAYER]:
                        env_state[ENV_STATUS_GAME] == 6
                        env_state[ENV_ID_ACTION] = env_state[ENV_TEMP_BUTTON]

                if env_state[ENV_STATUS_GAME] == 0:
                    env_state[ENV_STATUS_GAME] = 3
                elif env_state[ENV_STATUS_GAME] == 3:
                    env_state[ENV_STATUS_GAME] = 4
                elif env_state[ENV_STATUS_GAME] == 4:
                    env_state[ENV_STATUS_GAME] = 5
                elif env_state[ENV_STATUS_GAME] == 5:
                    env_state[ENV_STATUS_GAME] == 6


                if env_state[ENV_STATUS_GAME] == 6:
                    env_state = showdown(env_state)
                    env_state[ENV_ID_ACTION] = env_state[ENV_TEMP_BUTTON]
                    
            else:
                #cập nhật người chơi
                env_state[ENV_ID_ACTION] = id_action_next
    return env_state

